In [1]:
from time import sleep
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import json
import tabula
from datetime import datetime

In [2]:
# puxar tabela

pdfUrl = 'https://www.gov.br/anvisa/pt-br/setorregulado/regularizacao/medicamentos/medicamentos-de-referencia/arquivos/lista-a-incluidos-050124.pdf'

pdf = tabula.read_pdf(pdfUrl, pages="all", multiple_tables=False)
table = pdf[0]

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [3]:
# corrigi o nome das colunas da tabela
table.columns = table.iloc[0]

# remove a primeira linha da tabela
table = table[1:]

In [ ]:
# mostra tabela
display(table)

In [4]:
# verifica quantos dados estao errados

sus = err = 0
errList = []
for row in range(0, table.shape[0]):
    row = int(row)
    try:
        datetime.strptime(table.iloc[row, 6], "%d/%m/%Y").date()
        sus+=1
    except:
        err+=1
        errList.append(row)

display(f'err: {err} // sus: {sus}')
print('Erros encontrados nas linhas:', errList)

'err: 40 // sus: 1675'

Erros encontrados nas linhas: [30, 60, 61, 62, 63, 64, 65, 68, 74, 75, 76, 84, 87, 106, 107, 108, 111, 129, 130, 131, 141, 144, 145, 146, 178, 230, 234, 235, 241, 242, 243, 251, 252, 576, 577, 763, 764, 1108, 1213, 1214]


In [5]:
alvo = 144
print(len(table.iloc[alvo, 3]))
print(table.iloc[alvo, 6])
display(table[alvo-2:alvo+2])

8
nan


,FÁRMACO,DETENTOR,MEDICAMENTO,REGISTRO,CONCENTRAÇÃO,FORMA FARMACÊUTICA,DATA INCLUSÃO
143,azacitidina,United Medical,VIDAZA,125760020,100mg/4mL,pó liof sus inj,01/07/2014
144,azatioprina,Aspen Pharma,IMURAN,137640128,50 mg,com rev,12/11/2012
145,"azelaico, ácido",Leo Pharma,AZELAN,150 mg/g,gel,12/11/2012,NaN
146,"azelaico, ácido",Leo Pharma,AZELAN,200 mg/g,crem derm,12/11/2012,NaN


In [ ]:
# corrigi o cado de multivaroes em uma celula (registro)

for row in range(0, table.shape[0]):
    if len(table.iloc[row, 3]) > 9:
        regList = table.iloc[row, 3].split('\r')
        table.iloc[row, 3] =  regList[0]
        regList = regList[1:]
        nextRow = row + 1

        while (len(regList) > 0):
            for col in range(table.shape[1]-1, 2, -1):
                if col != 3:
                    table.iloc[nextRow, col] = table.iloc[nextRow, col-1]
                else:
                    table.iloc[nextRow, col] = regList[0]
                    regList = regList[1:]
            
            nextRow += 1

,FÁRMACO,DETENTOR,MEDICAMENTO,REGISTRO,CONCENTRAÇÃO,FORMA FARMACÊUTICA,DATA INCLUSÃO
1,abacavir (sulfato),Glaxosmithkline,ZIAGENAVIR,101070234,20 mg/mL,sol oral,30/11/2012
2,abacavir (sulfato),Glaxosmithkline,ZIAGENAVIR,101070234,300 mg,com rev,30/11/2012
3,abemaciclibe,Eli Lilly,VERZENIOS,112600199,50mg,com rev,10/02/2023
4,abemaciclibe,Eli Lilly,VERZENIOS,112600199,100mg,com rev,10/02/2023
5,abemaciclibe,Eli Lilly,VERZENIOS,112600199,150mg,com rev,10/02/2023
...,...,...,...,...,...,...,...
1711,zolpidem (hemitartarato),EMS Sigma Pharma,PATZ SL,135690643,10 mg,com subl,06/05/2022
1712,zolpidem (hemitartarato),Sanofi Medley,STILNOX,183260352,10 mg,com rev,12/11/2012
1713,zolpidem (hemitartarato),Sanofi-Aventis,STILNOX CR,113001001,"6,25 mg",com multicamada de lib\rprol,12/11/2012
1714,zolpidem (hemitartarato),Sanofi-Aventis,STILNOX CR,113001001,"12,5 mg",com multicamada de lib\rprol,12/11/2012
